### Run the following cell only once after kernel started

In [1]:
import os
os.chdir('../')

### Make sure you are in "ondewo-t2s-client-python" folder

In [2]:
os.getcwd()

'/home/aryskin/ondewo/ondewo-t2s'

In [3]:
import io
import soundfile as sf
import IPython.display as ipd
import grpc
from ondewo_grpc.ondewo.t2s import text_to_speech_pb2, text_to_speech_pb2_grpc
import google.protobuf.empty_pb2 as empty_pb2
from google.protobuf.json_format import ParseDict, MessageToDict, MessageToJson

### Set up the parameters of the grpc server. The example below is for the case when server is running locally

In [6]:
MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "localhost"
GRPC_PORT: str = "50655"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"

options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]


channel = grpc.insecure_channel(CHANNEL, options=options)
stab = text_to_speech_pb2_grpc.Text2SpeechStub(channel=channel)

### List all t2s pipelines present on the server

In [13]:
pipelines = stab.ListT2sPipelines(request=empty_pb2.Empty()).pipelines

### Select pipelines for specific language language

In [14]:
def find_pipeline_for_language(pipelines, language):
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

In [15]:
english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

In [16]:
pipelines

[id: "glow_tts&hifi_gan-e976dd6c-2f41-484b-aec2-3e6868d37280"
description {
  language: "de"
  speaker_sex: "female"
  pipeline_owner: "ondewo"
  comments: "trained on public domain dataset"
  speaker_name: "Kerstin"
  domain: "general"
}
active: true
inference {
  type: "composite"
  composite_inference {
    text2mel {
      type: "glow_tts"
      glow_tts {
        batch_size: 5
        use_gpu: true
        length_scale: 1.0
        noise_scale: 0.6669999957084656
        path: "models/glow-tts/de/kerstin_blank.pth"
        param_config_path: "models/glow-tts/de/config_blank.json"
      }
      glow_tts_triton {
        batch_size: 8
        length_scale: 1.0
        noise_scale: 0.6669999957084656
        max_text_length: 100
        param_config_path: "models/glow-tts/de/config_blank.json"
        triton_url: "localhost:50511"
        triton_model_name: "glow_tts"
      }
    }
    mel2audio {
      type: "hifi_gan"
      mb_melgan_triton {
        config_path: "models/mb_melgan/

### Make synthesize request to the server to get audio for given text

In [22]:
request = text_to_speech_pb2.SynthesizeRequest(text='magic_word', t2s_pipeline_id=german_pipeline.id, length_scale = 1.0, pcm=0, audio_format= 0)
response = stab.Synthesize(request=request)
print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio, )

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 1.6137868165969849 sec. Generation time is 0.8087550401687622 sec.


In [25]:
request = text_to_speech_pb2.SynthesizeRequest(text='Masada ist eine archäologische Stätte in Israel. Auf einem Gipfelplateau am Rand der Judäischen Wüste', t2s_pipeline_id=english_pipeline.id, length_scale = 1.0)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 8.562721252441406 sec. Generation time is 0.2562520503997803 sec.


### Adding length scale parameter to make speech faster or slower

In [11]:
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', t2s_pipeline_id=english_pipeline.id, length_scale=0.5)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 0.7778685092926025 sec. Generation time is 0.11872763186693192 sec.


### Get pipeline you want to update

In [35]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)
pipeline_config = stab.GetT2sPipeline(request=request)

### Change parameter in the pipeline config. For example default length scale

In [36]:
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 2

### Update pipeline

In [37]:
stab.UpdateT2sPipeline(request=pipeline_config)

### See if generated audio change according to updated config

In [38]:
request = text_to_speech_pb2.SynthesizeRequest(text='Hi, how are you?', t2s_pipeline_id=english_pipeline.id)
response = stab.Synthesize(request=request)

print(f'Length of the generated audio is {response.audio_length} sec.', f'Generation time is {response.generation_time} sec.')

bio = io.BytesIO(response.audio)

audio = sf.read(bio)

ipd.Audio(audio[0], rate=audio[1])

Length of the generated audio is 2.9605441093444824 sec. Generation time is 0.17088665068149567 sec.


### Change parameter back to previous (length_scale = 1.0)

In [39]:
request = text_to_speech_pb2.T2sPipelineId(id=english_pipeline.id)
pipeline_config = stab.GetT2sPipeline(request=request)
pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 1.0
stab.UpdateT2sPipeline(request=pipeline_config)

In [6]:
l = ['f', 'k']
from uuid import UUID

In [5]:
abs(hash(str(l)))

468457498735754727

In [13]:
UUID(bytes=b'kk')

ValueError: bytes is not a 16-char string

In [11]:
import torch


In [12]:
torch.__version__


'1.9.0+cu102'